In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm
import random
import pygraphviz
from pyvis.network import Network

### Modularidade



Uma das características de uma comunidade é que os vértices de uma dada comunidade devem ter uma tendência para uma maior número de ligações com outros membros da mesma comunidade que com membros de outras comunidades.

Uma boa medida de detecção de comunidades seria mensurar quão maior é o número de ligações internas entre membros de uma mesma comunidade em relação a uma rede gerada aleatoriamente.

Modularidade é equivalente ao coeficiente de assortatividade não normalizado para atributos categóricos, sendo que aqui os atributos categóricos identificam as comunidades aos quais os nós pertencem.

A modularidade de uma rede pode ser obtida se usando método **modularity()** do **networkx**.

O cálculo de modularidade é um problema muito complexo, assim heurísticas são usadas para o cálculo de modularidade de uma rede.
- algoritmo de **Clauset-Newman-Moore** 
- algoritmo de **Louvain**

**Algoritmo Clauset-Newman-Moore**

- Inicialmente é formado uma comunidade para cada vértice.
- Em cada passo, agregam-se duas comunidades cujo agrupamento produza o maior aumento do coeficiente de modularidade. 

Este algoritmo está implementado no método **greedy_modularity_communities()** do **networkx**.

**Algoritmo de Louvain**

Fase 1:

- Forma-se uma comunidade com cada vértice. 
- Para cada vértice, o algoritmo avalia o aumento de modularidade pela mudança para uma comunidade vizinha, até  que a mudança de vértices não produza aumento da modularidade.

Fase 2:
- Cada comunidade resultante da fase anterior é representada por um vértice no novo grafo.
- Agrega-se uma aresta entre dois vértices se as correspondentes comunidades estiverem ligadas. 
- O peso da aresta é igual à soma dos pesos das arestas que ligaram as comunidades na fase prévia.
- Arestas dentro de uma comunidade são representadas por um laço com peso igual à soma dos pesos das arestas originais.

As fases 1 e 2 são executadas até que nenhum nó na rede resultante mude para uma comunidade vizinha.

Este algoritmo está implementado no método **louvain_communities()** do **networkx**.

### Exemplo
Considere o grafo do Karate Clube

In [ ]:
KCG = nx.karate_club_graph()
fig, ax = plt.subplots(1,1,figsize=(15,10))
nx.draw(KCG, with_labels=True)

Encontrando comunidades usando o método **louvain_communities**

In [ ]:
louvain_comm = cm.louvain_communities(KCG,seed=123)
print(f"#comunidades: {len(louvain_comm)}")

for it in louvain_comm:
    print(f"{sorted(it)}")

Encontrando comunidades usando o método **greedy_modularity_communities**

In [ ]:
cmn_comm = cm.greedy_modularity_communities(KCG)
print(f"#cmn_comm: {len(cmn_comm)}")
for it in cmn_comm:
    print(sorted(it))

Calculando o coeficiente de modularidade de cada comunidade

In [ ]:
print(f"coef modularity louvain: {cm.modularity(KCG,louvain_comm)}")
print(f"coef modularity cmn: {cm.modularity(KCG,cmn_comm)}")

### Exemplo

Considere o exemplo da rede dos personagens do segundo livro da saga de Harry Potter.

In [ ]:
atri = pd.read_csv('../../data/harrypotter/hpattributes.txt', sep='\t')
ares = pd.read_csv('../../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../../data/harrypotter/hpnames.txt', sep='\t')

In [ ]:
gpotter = nx.DiGraph()

n = atri.shape[0]

for k in range(n):
    gpotter.add_node(k,
                     nome = nome['name'][k],
                     ano = atri['schoolyear'][k],
                     gen = atri['gender'][k],
                     casa = atri['house'][k])

for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

print(f"#nodes de G = {nx.number_of_nodes(gpotter)}")
print(f"#edges de G = {nx.number_of_edges(gpotter)}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
louvain_comm = cm.louvain_communities(gpotter)
cmn_comm = cm.greedy_modularity_communities(gpotter, cutoff=len(louvain_comm))

tam_louvain_comm = len(louvain_comm)
tam_cmn_comm = len(cmn_comm)

coef_louvain_comm = cm.modularity(gpotter,louvain_comm)
coef_cmn_comm = cm.modularity(gpotter,cmn_comm)

In [ ]:
print(f"#tam louvain comm = {len(louvain_comm)}")
print(f"#tam cmn comm = {len(cmn_comm)}")

In [ ]:
for it in louvain_comm:
    for k in it:
        print(f"{k}: {gpotter.nodes[k]['nome']}", end=", ")
    print()

In [ ]:
print(f"coef mod louvain: {coef_louvain_comm}")
print(f"coef mod cmn: {coef_cmn_comm}")

Definimos o mesmo número de comunidades geradas pelo algoritmo de **Louvain** no algoritmo de **Clauset-Newman-Moore** 

In [ ]:
gpotter_copy = gpotter.copy()

# matriz de adjacencia esparsa
A = nx.to_scipy_sparse_array(gpotter)#.toarray()
A_copy = A.copy()

# eliminando os nos isolados
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nos isolados.')

print(f"#nodes de G = {nx.number_of_nodes(gpotter)}")
print(f"#edges de G = {nx.number_of_edges(gpotter)}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

Para comparar os resultados, no algoritmo de **Clauset-Newman-Moore** só salvamos o mesmo número de comunidades que as devolvidas pelo algoritmo de **Louvain**. 

In [ ]:
louvain_comm = cm.louvain_communities(gpotter)
cmn_comm = cm.greedy_modularity_communities(gpotter, cutoff=len(louvain_comm))

tam_louvain_comm = len(louvain_comm)
tam_cmn_comm = len(cmn_comm)

coef_louvain_comm = cm.modularity(gpotter,louvain_comm)
coef_cmn_comm = cm.modularity(gpotter,cmn_comm)

In [ ]:
print(f"tam louvain comm = {len(louvain_comm)}")
print(f"tam cmn comm = {len(cmn_comm)}")

In [ ]:
print(f"coef mod louvain: {cm.modularity(gpotter,louvain_comm)}")
print(f"coef mod cmn: {cm.modularity(gpotter,cmn_comm)}")

In [ ]:
for it in louvain_comm:
    for k in it:
        print(f"{k}: {gpotter.nodes[k]['nome']}", end=", ")
    print()

In [ ]:
for it in cmn_comm:
    for k in it:
        print(f"{k}: {gpotter.nodes[k]['nome']}", end=", ")
    print()

In [ ]:
list_louvain_color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(tam_louvain_comm)]
print(len(list_louvain_color))

In [ ]:
list_cmn_color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(tam_cmn_comm)]
print(len(list_cmn_color))

In [ ]:
louvain_color = []
for no in gpotter.nodes():
    for k in range(len(louvain_comm)):
        if (no in louvain_comm[k]):
            louvain_color.append(list_louvain_color[k])

In [ ]:
cmn_color = []
for no in gpotter.nodes():
    for k in range(len(cmn_comm)):
        if (no in cmn_comm[k]):
            cmn_color.append(list_cmn_color[k])

In [ ]:
print(louvain_color)
print(cmn_color)

In [ ]:
etiquetas = nx.get_node_attributes(gpotter, 'nome')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 with_labels=True,
                 #labels=etiquetas,
                 font_size=10, 
                 node_color=louvain_color
                 )

plt.box(False)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 with_labels=True,
                 #labels=etiquetas,
                 font_size=10, 
                 node_color=cmn_color
                 )

plt.box(False)
plt.show()

### Exemplo

Considere o dataset **Stack Overflow Tag Network**, uma rede de tags do ***Stack Overflow*** baseada em histórias dos desenvolvedores.

Link: https://www.kaggle.com/datasets/stackoverflow/stack-overflow-tag-network



In [ ]:
df = pd.read_csv('../../data/stack_network/stack_network_links.csv')

In [ ]:
df.head()

In [ ]:
G = nx.Graph()

In [ ]:
with open('../../data/stack_network/stack_network_links.csv') as f:
  next(f)
  G = nx.parse_edgelist(f.readlines(), comments='/', delimiter=',', data=(('weight', float),))

In [ ]:
list(G.nodes())[:5]

In [ ]:
list(G.edges.data('weight'))[:5]

In [ ]:
cmn_comm = cm.greedy_modularity_communities(G, weight='weight')

In [ ]:
len(cmn_comm)

In [ ]:
for it in cmn_comm:
    print(it)

In [ ]:
plt.figure(figsize=(20,14))

nx.draw(G, pos = nx.nx_agraph.graphviz_layout(G), \
    node_size=200, node_color='lightblue', linewidths=1.5, \
    font_size=11, font_weight='bold', with_labels=True)

In [ ]:
comm_dict = {}

for i in range(len(cmn_comm)):
  for node in list(cmn_comm[i]):
    comm_dict[node] = i

nx.set_node_attributes(G, comm_dict, name='group')

In [ ]:
print(len(comm_dict))

In [ ]:
set_id_comm = set(comm_dict.values())
list_id_comm = list(set_id_comm)

In [ ]:
for id in list_id_comm:
    print(f"commmunity {id}:", end=" ")
    for key, value in comm_dict.items():
        if value == id:
            print(key, value, end=", ")
    print()

### Atividade

Encontre comunidades usando o algoritmo de **Louvain** e gere um gráfico mostrando o resultado obtido.

In [ ]:
net = Network(notebook=True, cdn_resources='in_line')
_ = net.from_nx(G)
_ = net.show('community_cmn.html')